In [115]:
import pandas as pd
from icecream import ic
import numpy as np
import random
from itertools import combinations
from geopy.distance import geodesic
#from itertools import iterrows

cities= pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
cities
ic("454")

ic| '454'


'454'

In [116]:

DIST_MATRIX= np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


for c1 in range(len(cities)):
    DIST_MATRIX[c1,c1]=np.inf
    #DIST_MATRIX[0,c1]=np.inf
    #DIST_MATRIX[c1,0]=np.inf


masked_dist_matrix = np.ma.masked_equal(DIST_MATRIX, np.inf)
min_index = np.ma.argmin(masked_dist_matrix[0])


ic(masked_dist_matrix[0].mask.all(),min_index)
#np.argmin(DIST_MATRIX[0])

cities.head()


ic| masked_dist_matrix[0].mask.all(): np.False_
    min_index: np.int64(7)


,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [117]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [118]:
def startgreedy(start):
   giro=[]
   costi=[]
   startpoint=globalstart=posto= start #np.int64(np.random.randint(len(cities)))
   giro.append(startpoint)
   copydist=DIST_MATRIX.copy()
   distances = DIST_MATRIX.copy()
   distances[:, posto] = np.inf
      
   for i in range(len(cities)):
      if(i==(len(cities))-1):
         posto=globalstart
      else:  
         posto=np.argmin(distances[startpoint])

      costo=copydist[posto][startpoint]
      costi.append(costo)

      giro.append(posto)
      distances[:, posto] = np.inf
      startpoint=posto

      
      

   #ic(giro,costi)
   return(tsp_cost(giro))






In [119]:
def startexp2(startpoint,distances,ended,near,giro):
    nearest_index=0
    if(ended==True):
        nearest_index=2
    
    second_near_index=nearest_index+1   #meti versos
    near[nearest_index]=np.ma.argmin(distances[startpoint]) #piu vicino
    distances[startpoint,near[nearest_index]] = np.ma.masked  #collego piu vicino ad origine 
    distances[near[nearest_index],startpoint] = np.ma.masked
    near[second_near_index]=np.ma.argmin(distances[startpoint])

    distance1=distances[startpoint,near[second_near_index]]  
    distance2=distances[near[second_near_index],near[nearest_index]]



    if(distance2<distance1):  #link start to 0 and 0 to  1
     # distances[near[nearest_index]] = np.ma.masked
      #distances[:,near[nearest_index]] = np.ma.masked

      distances[near[nearest_index],near[second_near_index]] = np.ma.masked
      distances[near[second_near_index],near[nearest_index]] = np.ma.masked
     
      giro[startpoint] = near[nearest_index]
      distances[startpoint]=np.ma.masked  # none can start from the startpoint 
      distances[:,near[nearest_index]]=np.ma.masked  # none can get to the second nearest 
      giro[near[nearest_index]] = near[second_near_index]
      distances[near[nearest_index]]=np.ma.masked  # none can start from the startpoint 
      distances[:,near[second_near_index]]=np.ma.masked  # none can get to the second nearest 
      ic("da66 " + str(startpoint) + " ad " + str(near[nearest_index]))
      ic("da " + str(near[nearest_index]) + " ad " + str(near[second_near_index]))
      near[second_near_index]=near[second_near_index]   #ils ecmondo piu vicino rimane uguale 
      near[nearest_index]=startpoint


    else:  # link start to 0 and start to 1
         distances[startpoint] = np.ma.masked
         distances[:, startpoint] = np.ma.masked
         distances[startpoint, near[second_near_index]] = np.ma.masked
         distances[near[second_near_index], startpoint] = np.ma.masked
         giro[near[second_near_index]] = startpoint
         giro[startpoint] = near[nearest_index]
         distances[near[second_near_index]]=np.ma.masked  # none can start from the second near 
         distances[:,near[nearest_index]]=np.ma.masked  # none can get to the  nearest 
         ic("da33 " + str(near[second_near_index]) + " ad " + str(startpoint))
         ic("da22 " + str(startpoint) + " ad " + str(near[nearest_index]))






def startexp3(startpoint,distances,ended,near,giro):
    nearest_index=0
    if(ended==True):
        nearest_index=2
    
    second_near_index=nearest_index+1   #meti versos
    near[nearest_index]=np.ma.argmin(distances[startpoint]) #piu vicino
    distances[startpoint,near[nearest_index]] = np.ma.masked  #collego piu vicino ad origine 
    distances[near[nearest_index],startpoint] = np.ma.masked
    near[second_near_index]=np.ma.argmin(distances[:,startpoint])

    

    distances[startpoint] = np.ma.masked
    distances[:, startpoint] = np.ma.masked
    distances[startpoint, near[second_near_index]] = np.ma.masked
    distances[near[second_near_index], startpoint] = np.ma.masked
    giro[near[second_near_index]] = startpoint
    giro[startpoint] = near[nearest_index]
    distances[near[second_near_index]]=np.ma.masked  # none can start from the second near 
    distances[:,near[nearest_index]]=np.ma.masked  # none can get to the  nearest 
    ic("da33 " + str(near[second_near_index]) + " ad " + str(startpoint))
    ic("da22 " + str(startpoint) + " ad " + str(near[nearest_index]))



In [133]:
def startExploring(startpoint,distances,near,giro,ended):
   rightindex= 0
   leftindex=1
   if ended==True:
      rightindex+=2
      leftindex+=2
   near[rightindex]=np.ma.argmin(distances[startpoint])
   distances[startpoint,near[rightindex]] = np.ma.masked
   distances[near[rightindex],startpoint] = np.ma.masked
   
   near[leftindex]=np.ma.argmin(distances[startpoint])
   distance1=distances[startpoint,near[leftindex]]
   distance2=distances[near[rightindex],near[leftindex]]
   if(distance2<distance1):  #link start to 0 and 0 to  1
      distances[near[rightindex]] = np.ma.masked
      distances[:,near[rightindex]] = np.ma.masked
      distances[near[rightindex],near[leftindex]] = np.ma.masked
      distances[near[leftindex],near[rightindex]] = np.ma.masked
      if(ended==True):
            
            
            giro[near[leftindex]] = near[rightindex]
            giro[near[rightindex]] = startpoint
         
         
         
            ic("da55 " + str(near[leftindex]  ) + " ad " + str(near[rightindex]))
            ic("da " + str( near[rightindex] ) + " ad " + str(startpoint  ))
            #near[leftindex]=near[rightindex]
            #near[rightindex]=startpoint
            
      else:
            giro[startpoint] = near[rightindex]
            giro[near[rightindex]] = near[leftindex]
            ic("da66 " + str(startpoint) + " ad " + str(near[rightindex]))
            ic("da " + str(near[rightindex]) + " ad " + str(near[leftindex]))
            near[leftindex]=near[rightindex]
            near[rightindex]=startpoint


   else:  # link start to 0 and start to 1
         distances[startpoint] = np.ma.masked
         distances[:, startpoint] = np.ma.masked
         distances[startpoint, near[leftindex]] = np.ma.masked
         distances[near[leftindex], startpoint] = np.ma.masked
         if ended == True:
            giro[startpoint] = near[leftindex]
            giro[near[rightindex]] = startpoint
            ic("da44 " + str(startpoint) + " ad " + str(near[leftindex]))
            ic("da11 " + str(near[rightindex]) + " ad " + str(startpoint))
         else:
            giro[near[leftindex]] = startpoint
            giro[startpoint] = near[rightindex]
            ic("da33 " + str(near[leftindex]) + " ad " + str(startpoint))
            ic("da22 " + str(startpoint) + " ad " + str(near[rightindex]))


def findNearToNear(near,distances,giro):
   togo=0
   count =len(giro)
   for e in giro:
      if e ==-1:
         togo+=1
   blocked=[]
   startvalue=-1-len(giro)

   neartonear=[startvalue,startvalue,startvalue,startvalue]
   n=0

   while n<3:
      onlybranch=0
      count=0
      if(n==2):
         count = np.count_nonzero(giro != -1)
      if(near[0+n]>=0 and near[1+n] >=0 ):

         distances[near[0+n],near[1+n]] = distances[near[1+n],near[0+n]]= np.ma.masked
 
      if near[0+n] >=0 and (not distances[near[0+n]].mask.all()):
         
         neartonear[0+n]=np.ma.argmin(distances[near[0+n]])
         #if(count>1):
         #   distances[near[0+n],neartonear[0+n]]=np.ma.masked
         #   neartonear[0+n]=np.ma.argmin(distances[near[0+n]])


      else:
         onlybranch+=1
         ic("rpof")
     
      #distances[near[1+n],neartonear[0+n]] =  distances[neartonear[0+n],near[1+n]]=np.ma.masked

      if   near[1+n] >=0 and  not distances[:,near[1+n]].mask.all():
         neartonear[1+n]=np.ma.argmin(distances[:,near[1+n]])
        
      else:
         onlybranch+=2
         ic("rpof")
      val=0
      if(onlybranch==0 and neartonear[1+n]==neartonear[n]):
         dist2=distances[near[1+n],neartonear[1+n]]
         dist1=distances[near[n],neartonear[n]]
         if(dist2>dist1):
            val=0
         else:
            val=1
      if(onlybranch!= 3 and onlybranch!=1):
         if (distances.mask[neartonear[n+val],near[n+val]]==False):
         #if (n+val)%2==0:
            giro[neartonear[n+val]]=near[n+val]
            if (checkCycle(giro,neartonear[val+n])==True):
               giro[neartonear[val+n]]=-1
               distances[neartonear[val+n],near[val+n]]=np.ma.masked
               neartonear[val+n]=np.ma.argmin(distances[:,near[val+n]])
               giro[neartonear[val+n]]=near[val+n]


            ic("da" + str(neartonear[n+val]) + " ad " + str(near[n+val]))
            distances[neartonear[n+val]]=np.ma.masked
            distances[:,near[n+val]]=np.ma.masked
         
         #else:
         elif (distances.mask[near[n+val],neartonear[n+val]]==False):
            giro[near[n+val]]=neartonear[n+val]
            if (checkCycle(giro,near[val+n])==True):
               giro[near[val+n]]=-1
               
               distances[near[val+n],neartonear[val+n]]=np.ma.masked
               neartonear[val+n]=np.ma.argmin(distances[near[val+n]])
               giro[near[val+n]]=neartonear[val+n]
            
            ic("da" + str(near[n+val]) + " ad " + str(neartonear[n+val]))
            distances[near[n+val]]=np.ma.masked
            distances[:,neartonear[n+val]]=np.ma.masked
         else:
             ic("erorre")

           
         
         distances[neartonear[n+val],near[n+val]]=np.ma.masked
         distances[near[n+val],neartonear[n+val]]=np.ma.masked

         ic(giro)

      #ic(np.ma.argmin(distances[near[1-val+n]]))
      if(onlybranch==0 and neartonear[1+n]==neartonear[n]):
         if not distances[near[1-val+n]].mask.all():
            neartonear[1-val+n]=np.ma.argmin(distances[near[1-val+n]])
         else:
            ic("rpof")
      if(onlybranch!= 3 and onlybranch!=2):
         if (distances.mask[neartonear[1-val+n],near[1-val+n]]==False):

            giro[neartonear[1-val+n]]=near[1-val+n]
            ic("da" + str(neartonear[1-val+n]) + " ad " + str(near[1-val+n]))


            if (checkCycle(giro,neartonear[1-val+n])==True):
               giro[neartonear[1-val+n]]=-1
               distances[neartonear[1-val+n],near[1-val+n]]=np.ma.masked
               neartonear[1-val+n]=np.ma.argmin(distances[:,near[1-val+n]])
               giro[neartonear[1-val+n]]=near[1-val+n]
               


            distances[neartonear[1-val+n]]=np.ma.masked
            distances[:,near[1-val+n]]=np.ma.masked
    
         elif (distances.mask[near[1-val+n],neartonear[1-val+n]]==False):

            giro[near[1-val+n]]=neartonear[1-val+n]
            if (checkCycle(giro,near[1-val+n])==True):
               giro[near[1-val+n]]=-1
               distances[near[1-val+n],neartonear[1-val+n]]=np.ma.masked
               neartonear[1-val+n]=np.ma.argmin(distances[near[1-val+n]])
               giro[near[1-val+n]]=neartonear[1-val+n]


            ic("da" + str(near[1-val+n]) + " ad " + str(neartonear[1-val+n]))
            distances[near[1-val+n]]=np.ma.masked
            distances[:,neartonear[1-val+n]]=np.ma.masked

         else:
             ic("erorre")
         #ic(giro)
         distances[neartonear[1-val+n],near[1-val+n]]=np.ma.masked
         distances[near[1-val+n],neartonear[1-val+n]]=np.ma.masked
      
         
         ic(giro)
      


         

      n+=2
   
   return neartonear
   

   




def checkNearCopy(near,distances,giro):  # controlla se è possibile un ciclo doppio 
   nequal=0
   nearest1=-len(giro)-1
   nearest2=-len(giro)-1


   if(near[0]==near[2]):  # fai in modo che sia puntato da near nearest deve avere ingeresso libero , near second nearest deve avere uscita livera
      nequal+=1
      nearest1=near[1]
      nearest2=near[3]
      nearest1=np.ma.argmin(distances[:,nearest1])
      nearest2=np.ma.argmin(distances[:,nearest2])
      
      

      

   elif (near[1]==near[2]):
      nequal+=1
      nearest1=near[0]
      nearest2=near[3]
      nearest1=np.ma.argmin(distances[nearest1])
      nearest2=np.ma.argmin(distances[:,nearest2])
   
   elif(near[0]==near[3]):
      nequal+=1
      nearest1=near[1]
      nearest2=near[2]
      nearest1=np.ma.argmin(distances[:,nearest1])
      nearest2=np.ma.argmin(distances[nearest2])
   
   elif(near[1]==near[3]):
      nequal+=1
      nearest1=near[0]
      nearest2=near[2]
      nearest1=np.ma.argmin(distances[nearest1])
      nearest2=np.ma.argmin(distances[nearest2])

   
   if(nequal>1):
      ic("gravissimo errore")
   
   if(nequal!=0):

      startexp3(nearest1,distances,False,near,giro)

      startexp3(nearest2,distances,True,near,giro)


   return 

def checkCycle(giro,start):
   togo=0
   step=giro[start]
   i=0
   while (step!=start and step>=0):
      i+=1
      step=giro[step]
      if step==-1:
         return False
   
   if(i<len(giro)-1):
      return True
   
   return False
   

   




def startgreedy2(start):
   giro=[-1] * len(cities)
   n_link= np.zeros(len(cities))
   girovec= np.zeros(len(cities))



   costi=[]
   distances = np.ma.masked_equal(DIST_MATRIX, np.inf)
   start= np.argmax(distances)
   end=np.argmax(distances[start])

   startpoint=globalstart=posto= start #np.int64(np.random.randint(len(cities)))
   
   
   copydist=DIST_MATRIX.copy()
   
   startvalue=-len(cities)
   near=[startvalue,startvalue,startvalue,startvalue]


   #end=np.ma.argmax(distances[startpoint])


   startexp3(startpoint,distances,False,near,giro)
   ic(giro)
   ic(near)
   
   startexp3(end,distances,True,near,giro)

   ic(giro)
   ic(near)

   #checkNearCopy(near,distances,giro)

   

   neartonear=findNearToNear(near,distances,giro)

   #checkNearCopy(near,distances,giro)
   
   ic(giro)
   


   near=neartonear.copy()
   ic(near)
   neartonear=findNearToNear(near,distances,giro)
   #checkNearCopy(near,distances,giro)
   near=neartonear.copy()
   ic(near)
   ic(giro)
   neartonear=findNearToNear(near,distances,giro)
   #checkNearCopy(near,distances,giro)

   
   
   
   ic(neartonear)
   ic(near)

   #stopping phase

   

   """
   near[0]=np.ma.argmin(distances[startpoint])
   distances[startpoint,near[0]] = np.ma.masked
   distances[near[0],startpoint] = np.ma.masked
   
   near[1]=np.ma.argmin(distances[startpoint])
   distance1=distances[startpoint,near[1]]
   distance2=distances[near[0],near[1]]
   if(distance2<distance1):  #link start to 0 and 0 to  1
      distances[near[0]] = np.ma.masked
      distances[:,near[0]] = np.ma.masked
      distances[near[0],near[1]] = np.ma.masked
      distances[near[1],near[0]] = np.ma.masked
      giro[startpoint]=near[0]
      giro[near[0]]=near[1]
      near[0]=near[1]
      near[1]=startpoint
   
   else:  #link start to 0 and start to 1
      distances[startpoint] = np.ma.masked
      distances[:,startpoint] = np.ma.masked
      distances[startpoint,near[1]] = np.ma.masked
      distances[near[1],startpoint] = np.ma.masked
      giro[near[1]]=startpoint
      giro[startpoint]=near[0]
   
   
   """
   ic(giro)





   
  

   #distances[:, posto] = np.inf





   
   """
   for i in range(len(cities)):
      if(i==(len(cities))-1):
         posto=globalstart
      else:  
         posto=np.argmin(distances[startpoint])
      
      n_link[posto]+=1
      n_link[startpoint]+=1
      


      costo=copydist[posto][startpoint]
      costi.append(costo)

      giro.append(posto)
      distances[:, posto] = np.inf
      startpoint=posto

      
      

   
   ic(giro)
   return(tsp_cost(giro))
   """  

   return giro


        
    
#giro0=startgreedy2(1)

#ic(giro0)

In [121]:
def devround(path,start):
    newpath=[]
    newpath.append(start)
    for e in range(len(path)):
        start=path[start]
        newpath.append(start)
   
    ic(newpath)
    return newpath

In [122]:
np.argmin(DIST_MATRIX[0])

np.int64(7)

In [134]:
totalcost=0
#ic(tsp_cost(devround(startgreedy2(2),0)))
#ic(tsp_cost(devround(startgreedy2(0),0)))
#ic(startgreedy(0))

for i in range(len(cities)):
    totalcost+=tsp_cost(devround(startgreedy2(i),i))
    totalcost-=startgreedy(i)
    #ic(tsp_cost(devround(startgreedy2(i),i)))
 
    #ic(startgreedy(i))
    #ic(totalcost)
ic(totalcost)

ic| "da33 " + str(near[second_near_index]) + " ad " + str(startpoint): 'da33 2 ad 6'
ic| "da22 " + str(startpoint) + " ad " + str(near[nearest_index]): 'da22 6 ad 5'
ic| giro: [-1, -1, np.int64(6), -1, -1, -1, np.int64(5), -1]
ic| near: [np.int64(5), np.int64(2), -8, -8]
ic| "da33 " + str(near[second_near_index]) + " ad " + str(startpoint): 'da33 1 ad 0'
ic| "da22 " + str(startpoint) + " ad " + str(near[nearest_index]): 'da22 0 ad 7'
ic| giro: [np.int64(7), np.int64(0), np.int64(6), -1, -1, -1, np.int64(5), -1]
ic| near: [np.int64(5), np.int64(2), np.int64(7), np.int64(1)]
ic| "da" + str(neartonear[n+val]) + " ad " + str(near[n+val]): 'da3 ad 2'
ic| giro: [np.int64(7), np.int64(0), np.int64(6), np.int64(2), -1, -1, np.int64(5), -1]
ic| "da" + str(near[1-val+n]) + " ad " + str(neartonear[1-val+n]): 'da5 ad 4'
ic| giro: [np.int64(7),
           np.int64(0),
           np.int64(6),
           np.int64(2),
           -1,
           np.int64(4),
           np.int64(5),
           -1]
ic| "d

np.float64(-488.42188294671223)